In [ ]:
import numpy as np
import nibabel as nib
import PIL
import os
import sys
import matplotlib.pyplot as plt

In [ ]:
nifti_file_path = '../../KiTS23/KiTS_subset/case_00016/imaging.nii.gz'
segmentations_file_path = '../../KiTS23/KiTS_subset/case_00016/segmentation.nii.gz'

img = nib.load(nifti_file_path)
seg = nib.load(segmentations_file_path)

print(img)

In [ ]:
print(seg)

In [ ]:
viz_file_path = nifti_file_path
# viz_file_path = segmentations_file_path
idx = -1
# idx = 55

try:
    # Load the NIfTI image
    img = nib.load(viz_file_path)

    # Get the image data as a NumPy array
    img_data = img.get_fdata()
    if idx < 0:
        idx = img_data.shape[0] // 2 + 1
    print(idx)

    # Get the shape of the data (e.g., (X, Y, Z) for 3D, or (X, Y, Z, T) for 4D)
    print(f"Image data shape: {img_data.shape}")

    # Select a slice to visualize (e.g., the middle slice along the Z-axis)
    # Adjust the axis and slice index based on your data and desired view
    if img_data.ndim == 3:
        mid_slice_z = img_data[:, :, img_data.shape[2] // 2]
#         mid_slice_z = img_data[img_data.shape[0] // 2, :, :]
        plt.imshow(mid_slice_z.T, cmap='gray', origin='lower')
        plt.title(f'Middle Slice (Z-axis)')
    elif img_data.ndim == 4:
        # If it's 4D (e.g., fMRI with time), pick a time point
        time_point = 0
        mid_slice_z = img_data[:, :, img_data.shape[2] // 2, time_point]
        plt.imshow(mid_slice_z.T, cmap='gray', origin='lower')
        plt.title(f'Middle Slice (Z-axis), Time Point {time_point}')
    else:
        print("Image dimension not supported for simple 2D slice visualization.")
        exit()

    plt.xlabel('First axis')
    plt.ylabel('Second axis')
    plt.colorbar(label='Signal Intensity')
    plt.show()

    # You can also visualize slices along other axes
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Axial slice (z-axis)
    axes[0].imshow(img_data[:, :, img_data.shape[2] // 2].T, cmap='gray', origin='lower')
    axes[0].set_title('Axial Slice')
    axes[0].axis('off')

    # Coronal slice (y-axis)
    axes[1].imshow(img_data[:, img_data.shape[1] // 2, :].T, cmap='gray', origin='lower')
    axes[1].set_title('Coronal Slice')
    axes[1].axis('off')

    # Sagittal slice (x-axis)
#     axes[2].imshow(img_data[img_data.shape[0] // 2, :, :].T, cmap='gray', origin='lower')
    axes[2].imshow(img_data[idx, :, :].T, cmap='gray', origin='lower')
    axes[2].set_title('Sagittal Slice')
    axes[2].axis('off')

    plt.suptitle('Orthogonal Slices')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust for suptitle
    plt.show()

except FileNotFoundError:
    print(f"Error: File not found at {nifti_file_path}")
except Exception as e:
    print(f"An error occurred: {e}")
    
    
viz_file_path = segmentations_file_path

try:
    # Load the NIfTI image
    img = nib.load(viz_file_path)

    # Get the image data as a NumPy array
    img_data = img.get_fdata()

    # Get the shape of the data (e.g., (X, Y, Z) for 3D, or (X, Y, Z, T) for 4D)
    print(f"Image data shape: {img_data.shape}")

    # Select a slice to visualize (e.g., the middle slice along the Z-axis)
    # Adjust the axis and slice index based on your data and desired view
    if img_data.ndim == 3:
#         mid_slice_z = img_data[:, :, img_data.shape[2] // 2]
        mid_slice_z = img_data[img_data.shape[0] // 2, :, :]
        plt.imshow(mid_slice_z.T, cmap='gray', origin='lower')
        plt.title(f'Middle Slice (Z-axis)')
    elif img_data.ndim == 4:
        # If it's 4D (e.g., fMRI with time), pick a time point
        time_point = 0
        mid_slice_z = img_data[:, :, img_data.shape[2] // 2, time_point]
        plt.imshow(mid_slice_z.T, cmap='gray', origin='lower')
        plt.title(f'Middle Slice (Z-axis), Time Point {time_point}')
    else:
        print("Image dimension not supported for simple 2D slice visualization.")
        exit()

    plt.xlabel('First axis')
    plt.ylabel('Second axis')
    plt.colorbar(label='Signal Intensity')
    plt.show()

    # You can also visualize slices along other axes
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Axial slice (z-axis)
    axes[0].imshow(img_data[:, :, img_data.shape[2] // 2].T, cmap='gray', origin='lower')
    axes[0].set_title('Axial Slice')
    axes[0].axis('off')

    # Coronal slice (y-axis)
    axes[1].imshow(img_data[:, img_data.shape[1] // 2, :].T, cmap='gray', origin='lower')
    axes[1].set_title('Coronal Slice')
    axes[1].axis('off')

    # Sagittal slice (x-axis)
    axes[2].imshow(img_data[img_data.shape[0] // 2, :, :].T, cmap='gray', origin='lower')
#     axes[2].imshow(img_data[idx, :, :].T, cmap='gray', origin='lower')
    axes[2].set_title('Sagittal Slice')
    axes[2].axis('off')

    plt.suptitle('Orthogonal Slices')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust for suptitle
    plt.show()

except FileNotFoundError:
    print(f"Error: File not found at {nifti_file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

